In [1]:
import pandas as pd
import numpy as np
import sqlite3
from pandasql import sqldf
import matplotlib.pyplot as plt
import statistics 
from scipy import stats

In [2]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [3]:
import dask.dataframe as dd


In [31]:
filepath = r"C:\Users\12155\Documents\Python\Capstone\mh_final_export.csv"
dsk_df = dd.read_csv(filepath, low_memory = False).set_index('CASEID')
dsk_df = dsk_df.compute()

In [32]:
dsk_df.head()

,YEAR,NUMBER_OF_DIAGNOSES,TRAUMA_STRESS_DISORDER,ANXIETY_DIS,ADHD_DIS,CONDUCT_DIS,DIMENTIA_DIL_DIS,BIPOLAR_DIS,DEPRESSIVE_DIS,OPPOSITIONAL_DEFIANT_DIS,...,marital_status,SMI_SED,SubstanceU_Prob,employment_status,not_in_labor_force_detailed,vet_status,residential_at_discharge_end,reporting_state,census_division,census_region
CASEID,,,,,,,,,,,,,,,,,,,,,
20200000002,2020,3,0,1,1,0,0,0,0,0,...,Never married,SED or at risk,No,Undefined,Undefined,Undefined,Undefined,Alabama,East South Central,South
20200000012,2020,1,0,0,0,0,0,0,1,0,...,Never married,Not SMI/SED,No,Undefined,Undefined,Undefined,Undefined,Alabama,East South Central,South
20200000017,2020,3,1,1,0,0,0,0,0,0,...,Undefined,SED or at risk,No,Undefined,Undefined,Undefined,Private residence,Alabama,East South Central,South
20200000019,2020,1,0,0,0,0,0,0,1,0,...,Never married,SED or at risk,No,Undefined,Undefined,No,Private residence,Alabama,East South Central,South
20200000028,2020,3,0,0,1,0,0,1,0,0,...,Never married,SED or at risk,No,Undefined,Undefined,No,Private residence,Alabama,East South Central,South


In [45]:
dsk_df = dsk_df.reset_index()

In [46]:
dsk_df.columns

Index(['CASEID', 'YEAR', 'NUMBER_OF_DIAGNOSES', 'TRAUMA_STRESS_DISORDER',
       'ANXIETY_DIS', 'ADHD_DIS', 'CONDUCT_DIS', 'DIMENTIA_DIL_DIS',
       'BIPOLAR_DIS', 'DEPRESSIVE_DIS', 'OPPOSITIONAL_DEFIANT_DIS',
       'PDEVELOPMENTAL_DIS', 'PERSONALITY_DIS',
       'SCHIZOPHRENIA_or_PSYCHOTIC_DIS', 'ALC_SUB_DIS', 'OTHER_DIS',
       'age_range', 'educ_category', 'ethnicity_category', 'race_label', 'sex',
       'state_psych_hosp', 'SMHAfundop_COMM_PROGRAM', 'served_other_psych_inp',
       'res_treat', 'under_justice_system', 'mh_diag_1', 'mh_diag_2',
       'mh_diag_3', 'SUD', 'marital_status', 'SMI_SED', 'SubstanceU_Prob',
       'employment_status', 'not_in_labor_force_detailed', 'vet_status',
       'residential_at_discharge_end', 'reporting_state', 'census_division',
       'census_region'],
      dtype='object')

Understanding co-occurence in diagnoses

In [49]:
mh = dsk_df

In [7]:
mh_sub = mh[['TRAUMA_STRESS_DISORDER', 'ANXIETY_DIS',
       'ADHD_DIS', 'CONDUCT_DIS', 'DIMENTIA_DIL_DIS', 'BIPOLAR_DIS',
       'DEPRESSIVE_DIS', 'OPPOSITIONAL_DEFIANT_DIS', 'PDEVELOPMENTAL_DIS',
       'PERSONALITY_DIS', 'SCHIZOPHRENIA_or_PSYCHOTIC_DIS', 'ALC_SUB_DIS',
       'OTHER_DIS']]

In [8]:
mh_sub.head()

,TRAUMA_STRESS_DISORDER,ANXIETY_DIS,ADHD_DIS,CONDUCT_DIS,DIMENTIA_DIL_DIS,BIPOLAR_DIS,DEPRESSIVE_DIS,OPPOSITIONAL_DEFIANT_DIS,PDEVELOPMENTAL_DIS,PERSONALITY_DIS,SCHIZOPHRENIA_or_PSYCHOTIC_DIS,ALC_SUB_DIS,OTHER_DIS
0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,1,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0


In [9]:
mh_chisquare = mh_sub.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)
mh_chisquare

,TRAUMA_STRESS_DISORDER,ANXIETY_DIS,ADHD_DIS,CONDUCT_DIS,DIMENTIA_DIL_DIS,BIPOLAR_DIS,DEPRESSIVE_DIS,OPPOSITIONAL_DEFIANT_DIS,PDEVELOPMENTAL_DIS,PERSONALITY_DIS,SCHIZOPHRENIA_or_PSYCHOTIC_DIS,ALC_SUB_DIS,OTHER_DIS
TRAUMA_STRESS_DISORDER,1.000000,0.135635,-0.171319,-0.083845,-0.011298,-0.039868,-0.128473,0.107968,-0.089879,-0.007127,-0.026170,-0.014849,-0.143190
ANXIETY_DIS,0.135635,1.000000,0.064328,0.059369,0.006853,0.004405,-0.089409,-0.064955,0.030277,-0.004836,0.002977,0.011170,0.055979
ADHD_DIS,-0.171319,0.064328,1.000000,-0.003868,-0.011504,0.011385,-0.132656,-0.139680,0.017222,-0.010381,-0.012341,-0.030377,-0.017732
CONDUCT_DIS,-0.083845,0.059369,-0.003868,1.000000,-0.004264,-0.007310,-0.062520,0.025237,-0.019754,-0.003049,-0.006804,0.024157,-0.034777
DIMENTIA_DIL_DIS,-0.011298,0.006853,-0.011504,-0.004264,1.000000,-0.000369,-0.007145,0.006326,-0.002432,-0.000603,-0.000283,-0.000477,-0.004274
BIPOLAR_DIS,-0.039868,0.004405,0.011385,-0.007310,-0.000369,1.000000,-0.026003,-0.004134,-0.003194,0.023942,0.016191,0.011979,-0.013782
DEPRESSIVE_DIS,-0.128473,-0.089409,-0.132656,-0.062520,-0.007145,-0.026003,1.000000,0.068575,-0.063680,0.015821,-0.003680,0.021729,-0.103202
OPPOSITIONAL_DEFIANT_DIS,0.107968,-0.064955,-0.139680,0.025237,0.006326,-0.004134,0.068575,1.000000,0.028345,0.003880,0.010000,-0.005895,0.036416
PDEVELOPMENTAL_DIS,-0.089879,0.030277,0.017222,-0.019754,-0.002432,-0.003194,-0.063680,0.028345,1.000000,-0.004691,0.002118,-0.015118,0.041720
PERSONALITY_DIS,-0.007127,-0.004836,-0.010381,-0.003049,-0.000603,0.023942,0.015821,0.003880,-0.004691,1.000000,0.003208,-0.000161,-0.000199


In [23]:
mh_chisquare.shape

(13, 13)

In [28]:
mh_chisquare.to_csv(r"C:\Users\12155\Documents\Python\Capstone\mh_chisquare.csv")

In [10]:
from scipy.stats import chi2_contingency

In [11]:
df = mh_sub

In [16]:
#understanding co-occurance of disorders
v = df.values
matrix_product = pd.DataFrame(v.T.dot(v), index=df.columns, columns=df.columns)
#https://stackoverflow.com/questions/50279402/compute-co-occurrence-matrix-by-counting-values-in-cells

In [17]:
matrix_product.to_csv(r"C:\Users\12155\Documents\Python\Capstone\matrix_product.csv")

Understading indexing of cases vs. population

In [53]:
#count distinct of caseid by state to understand total number of cases within each state 
state_case =  pd.DataFrame(mh.groupby(['reporting_state']).agg({'CASEID':'count'}).reset_index())

In [54]:
state_case.head()
#matches what I had previously compiled in Tableau

,reporting_state,CASEID
0,Alabama,31931
1,Alaska,3631
2,Arizona,121843
3,Arkansas,22734
4,California,241763


In [55]:
check = pd.DataFrame(state_case.agg({'CASEID':'sum'}))
check
#produces correct total number of cases

,0
CASEID,1915199


In [62]:
state_case.to_csv(r"C:\Users\12155\Documents\Python\Capstone\state_cases.csv")

In [64]:
#looking at only cases with a diagnosis, filter to only cases where MH1 is not "Undefined"
filtered_mh = mh[mh['mh_diag_1'] != 'Undefined']


In [65]:
check2 = pd.DataFrame(filtered_mh.agg({'CASEID':'count'}))
check2

,0
CASEID,1637967


In [66]:
state_case_w_Diag =  pd.DataFrame(filtered_mh.groupby(['reporting_state']).agg({'CASEID':'count'}).reset_index())
state_case_w_Diag

,reporting_state,CASEID
0,Alabama,31918
1,Alaska,3570
2,Arizona,102948
3,Arkansas,22122
4,California,225691
5,Colorado,35612
6,Connecticut,1144
7,D.C.,2989
8,Delaware,728
9,Florida,47238


In [67]:
check3 = pd.DataFrame(state_case_w_Diag.agg({'CASEID':'sum'}))

In [68]:
check3

,0
CASEID,1637967


In [69]:
mh['mh_diag_1'].value_counts()['Undefined']

277232

In [70]:
277232+1637967
#correct sum

1915199

In [71]:
state_case_w_Diag.to_csv(r"C:\Users\12155\Documents\Python\Capstone\state_case_w_Diag.csv")